In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt

In [51]:
def compute_relative_importance(X, y):
    model = LinearRegression().fit(X, y)
    importance_scores = pd.DataFrame(index=X.columns, columns=['R2 Drop', 'Correlation Squared', 'Beta Squared', 'Pratt Index'])
    
    baseline_r2 = r2_score(y, model.predict(X))
    for feature in X.columns:
        X_temp = X.drop(columns=[feature])
        model_temp = LinearRegression().fit(X_temp, y)
        r2_drop = baseline_r2 - r2_score(y, model_temp.predict(X_temp))
        importance_scores.at[feature, 'R2 Drop'] = r2_drop


    return importance_scores

In [52]:
def calculate_adjusted_r_squared(features, target):
    cumulative_r_squared = []
    for i in range(1, features.shape[1] + 1):
        current_features = features.iloc[:, :i]
        model = LinearRegression().fit(current_features, target)
        r_squared = model.score(current_features, target)
        cumulative_r_squared.append(r_squared)
    return cumulative_r_squared

In [31]:
data = pd.read_csv('test_sample.csv')

In [32]:
Y = data['Y']
X = data.drop('Y', axis=1)

In [53]:
def find_min_features_for_target_r2(features, target, target_r2=0.9):
    num_features = features.shape[1]
    for i in range(1, num_features + 1):
        model = LinearRegression().fit(features.iloc[:, :i], target)
        r_squared = model.score(features.iloc[:, :i], target)
        if r_squared >= target_r2:
            return i, r_squared
    # If target_r2 is not met, return stats with all features
    return num_features, r_squared

In [54]:
n_orig, achieved_r2 = find_min_features_for_target_r2(X, Y)

In [55]:
n_orig

359

In [56]:
achieved_r2

0.9000219053221528

In [57]:
def calculate_feature_importance(features, target):
    feature_names = features.columns
    
    # Fit a model with all features to compute the baseline R-squared
    full_model = sm.OLS(target, sm.add_constant(features)).fit()
    
    # Initialize a Series to store delta R-squared values
    delta_r_squared = pd.Series(index=feature_names)
    
    for feature in feature_names:
        # Drop one feature at a time and refit the model to see its impact
        reduced_features = features.drop(columns=feature)
        reduced_model = sm.OLS(target, sm.add_constant(reduced_features)).fit()
        delta_r_squared[feature] = full_model.rsquared - reduced_model.rsquared
    
    # Compute squared correlation between each feature and the target
    squared_correlation = features.apply(lambda x: np.corrcoef(target, x)[0,1] ** 2)
    
    # Calculate standard deviations for normalization
    std_features = features.std()
    std_target = np.std(target)
    
    # Compute Beta squared and Pratt Index using the full model coefficients
    beta_squared = (full_model.params[1:] * std_features / std_target) ** 2
    pratt_index = beta_squared * np.sqrt(squared_correlation)
    
    # Compile the results into a DataFrame
    results = pd.DataFrame({
        'DeltaR2': delta_r_squared,
        'SquaredCorrelation': squared_correlation,
        'BetaSquared': beta_squared,
        'PrattIndex': pratt_index
    }, index=feature_names)
    
    return results

In [38]:
def rSquar(j, y, X) :
    return sm.OLS(y, sm.add_constant(X.iloc[:,:j])).fit().rsquared

In [58]:
for num_features in range(1, len(X.columns) + 1):
        model_fit = sm.OLS(Y, X.iloc[:, :num_features]).fit()
        achieved_rsquared = model_fit.rsquared
        
        if achieved_rsquared > 0.9:
            # Subtract one to align with the original logic if needed
            n_features = num_features - 1
            print("Number of features (n_orig), R-squared:")
            print(n_features, achieved_rsquared)
            

Number of features (n_orig), R-squared:
360 0.9003563182027753
Number of features (n_orig), R-squared:
361 0.90109725472028
Number of features (n_orig), R-squared:
362 0.9015276488586625
Number of features (n_orig), R-squared:
363 0.9025595806062375
Number of features (n_orig), R-squared:
364 0.9025878672010089
Number of features (n_orig), R-squared:
365 0.9060944667376561
Number of features (n_orig), R-squared:
366 0.9061395759580629
Number of features (n_orig), R-squared:
367 0.9119780378385518
Number of features (n_orig), R-squared:
368 0.9129655723728975
Number of features (n_orig), R-squared:
369 0.9146635884232543
Number of features (n_orig), R-squared:
370 0.9147001668497157
Number of features (n_orig), R-squared:
371 0.9186031560287078
Number of features (n_orig), R-squared:
372 0.9187074994055655
Number of features (n_orig), R-squared:
373 0.9218126014934773
Number of features (n_orig), R-squared:
374 0.92307540847702
Number of features (n_orig), R-squared:
375 0.9230849688668

In [40]:
nFactors = 491

xPCA = PCA(n_components=nFactors)
xPCA.fit(X.iloc[:,:491])
xPCA_importance = pd.DataFrame({'Standard deviation': np.sqrt(xPCA.explained_variance_),
                               'Proportion of Variance': xPCA.explained_variance_ratio_,
                               'Cumulative Proportion': np.cumsum(xPCA.explained_variance_ratio_)},
                               columns=['Standard deviation','Proportion of Variance','Cumulative Proportion'],
                               index=[ "PC%i" %(j+1) for j in range(nFactors)])
print(xPCA_importance.T)

                             PC1       PC2       PC3       PC4       PC5  \
Standard deviation      3.971931  3.942693  3.885659  3.861962  3.834083   
Proportion of Variance  0.008030  0.007912  0.007685  0.007592  0.007482   
Cumulative Proportion   0.008030  0.015942  0.023627  0.031219  0.038701   

                             PC6       PC7       PC8       PC9      PC10  ...  \
Standard deviation      3.831501  3.784526  3.773444  3.766837  3.744334  ...   
Proportion of Variance  0.007472  0.007290  0.007248  0.007222  0.007136  ...   
Cumulative Proportion   0.046173  0.053463  0.060711  0.067933  0.075069  ...   

                           PC482     PC483     PC484     PC485     PC486  \
Standard deviation      0.079477  0.074710  0.069897  0.058444  0.049746   
Proportion of Variance  0.000003  0.000003  0.000002  0.000002  0.000001   
Cumulative Proportion   0.999988  0.999991  0.999993  0.999995  0.999996   

                           PC487         PC488         PC489     

In [41]:
factorLoadings = pd.DataFrame(xPCA.components_,
                              columns=["X%i" %(j+1) for j in range(491)],
                              index=["PC%i" %(j+1) for j in range(nFactors)])
factorScores = pd.DataFrame(np.dot(X.iloc[:,:491], xPCA.components_.T), columns =["PC%i"%(j+1) for j in range(nFactors)])
zeroLoading = xPCA.mean_

In [42]:
m491_PCA = sm.OLS(Y, sm.add_constant(factorScores)).fit()
print(m491_PCA.summary())

                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                     6951.
Date:                Sun, 07 Apr 2024   Prob (F-statistic):           4.68e-15
Time:                        23:53:29   Log-Likelihood:                 393.15
No. Observations:                 500   AIC:                             197.7
Df Residuals:                       8   BIC:                             2271.
Df Model:                         491                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.9179      0.363      2.528      0.0

In [44]:
metrics_PCA = compute_relative_importance(factorScores, Y)
print(metrics_PCA)

               last         first        betasq         pratt
PC1    1.409386e-03  1.409386e-03  1.412210e-03  1.410797e-03
PC2    2.999159e-03  2.999159e-03  3.005169e-03  3.002163e-03
PC3    7.770270e-03  7.770270e-03  7.785842e-03  7.778052e-03
PC4    5.223133e-03  5.223133e-03  5.233600e-03  5.228364e-03
PC5    1.342557e-02  1.342557e-02  1.345248e-02  1.343902e-02
...             ...           ...           ...           ...
PC487  1.864370e-06  1.864370e-06  1.868106e-06  1.866237e-06
PC488  2.378022e-06  2.378022e-06  2.382788e-06  2.380404e-06
PC489  4.647061e-07  4.647061e-07  4.656374e-07  4.651715e-07
PC490  4.981739e-08  4.981739e-08  4.991723e-08  4.986729e-08
PC491  3.266348e-07  3.266348e-07  3.272894e-07  3.269620e-07

[491 rows x 4 columns]


In [46]:
first_PCA_rank = metrics_PCA["first"].rank(ascending=False, method='first')
print(first_PCA_rank)

PC1      143.0
PC2       88.0
PC3       36.0
PC4       56.0
PC5       12.0
         ...  
PC487    460.0
PC488    455.0
PC489    471.0
PC490    486.0
PC491    475.0
Name: first, Length: 491, dtype: float64


In [47]:
metrics_PCA_sort = pd.DataFrame({"Factors" : first_PCA_rank.index,
                                 "Rank" : first_PCA_rank.values}).sort_values(by="Rank")
print(list(metrics_PCA_sort["Factors"]))

['PC53', 'PC52', 'PC46', 'PC17', 'PC56', 'PC36', 'PC156', 'PC30', 'PC19', 'PC22', 'PC178', 'PC5', 'PC247', 'PC81', 'PC31', 'PC9', 'PC99', 'PC192', 'PC15', 'PC213', 'PC51', 'PC12', 'PC221', 'PC202', 'PC98', 'PC114', 'PC59', 'PC64', 'PC13', 'PC91', 'PC39', 'PC27', 'PC137', 'PC197', 'PC35', 'PC3', 'PC102', 'PC195', 'PC83', 'PC49', 'PC216', 'PC146', 'PC77', 'PC107', 'PC21', 'PC68', 'PC273', 'PC103', 'PC209', 'PC134', 'PC115', 'PC237', 'PC218', 'PC44', 'PC133', 'PC4', 'PC165', 'PC54', 'PC219', 'PC173', 'PC147', 'PC143', 'PC230', 'PC259', 'PC38', 'PC307', 'PC240', 'PC182', 'PC117', 'PC6', 'PC37', 'PC308', 'PC109', 'PC28', 'PC163', 'PC251', 'PC16', 'PC314', 'PC121', 'PC67', 'PC89', 'PC232', 'PC29', 'PC189', 'PC242', 'PC151', 'PC270', 'PC2', 'PC361', 'PC130', 'PC171', 'PC234', 'PC10', 'PC95', 'PC42', 'PC86', 'PC181', 'PC148', 'PC72', 'PC129', 'PC198', 'PC323', 'PC286', 'PC40', 'PC34', 'PC149', 'PC211', 'PC329', 'PC63', 'PC207', 'PC82', 'PC185', 'PC371', 'PC47', 'PC168', 'PC23', 'PC118', 'PC210

In [48]:
orderedFactors = pd.DataFrame(factorScores, columns= metrics_PCA_sort["Factors"])

In [49]:
orderedLoadings = pd.DataFrame(factorLoadings.T,  columns= metrics_PCA_sort["Factors"])
orderedPCA_R2 = [rSquar(j,Y, orderedFactors) for j in range(2,491)]
print(orderedPCA_R2)

[0.07553207709672782, 0.10837916954112337, 0.14015300373357953, 0.16462206117530964, 0.1872481982108749, 0.20679577730977272, 0.224470884865838, 0.2410375978321232, 0.25516118702293245, 0.26906145116255853, 0.2824870226385696, 0.29584373573807854, 0.30881534963657886, 0.32086910144475733, 0.33254097546639183, 0.343896260326253, 0.35505054470559105, 0.36618222736681816, 0.3772884124307645, 0.38815259408328295, 0.3988712393272891, 0.40934909949736, 0.419741391857576, 0.4298670944810059, 0.4396861615856551, 0.44932492562490445, 0.4588000353161348, 0.4679217140551025, 0.4764687130477143, 0.48477528933397507, 0.4930444462790736, 0.5009899182834606, 0.5087708138549731, 0.5165500662430309, 0.5243203366269049, 0.5320297638256605, 0.5396948687672263, 0.5472286219574815, 0.5547231641363226, 0.5622131794413138, 0.569403187138994, 0.5764347237692606, 0.5834434517002147, 0.590180811153365, 0.5965482564351868, 0.602760536806751, 0.6088596266845341, 0.6146505455088598, 0.6203199922779179, 0.625918478

In [50]:
target_rsquared = 0.9
n_PCA = 0
for value in range(0,491):
  if orderedPCA_R2[value] > target_rsquared:
    n_PCA = value
    model_dimension_reduction = n_orig - n_PCA
    print ("n_orig, n_PCA, model dimension reduction, determination coefficient")
    print (n_orig,n_PCA,"%.5f"%model_dimension_reduction,"%.5f"%orderedPCA_R2[value])
    break

n_orig, n_PCA, model dimension reduction, determination coefficient
360 148 212.00000 0.90022
